# Testing of te networks desing and training process

## 0. importing and preprocessing of the data

In [19]:
import pandas as pd
import torch
from transformers import(
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments)
from training_utils import tokenize_and_position_sequence, baseDataset
from relation_extraction_encoders_modeling import Classifier_MLP
from torchinfo import summary
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [20]:
data_path = '/home/tensorboard/Documentos/1. D4R/Relation extraction/test_20-06-24.csv'
custom_model_path = '/home/tensorboard/Documentos/1. D4R/Relation extraction/custom basic model/basic_model'
custom_tokenizer_path = '/home/tensorboard/Documentos/1. D4R/Relation extraction/custom basic model/custom tokenizer'

In [21]:
data = pd.read_csv(data_path)

In [22]:
data

,Unnamed: 0,Sentence,Label
0,0,Doña Ana Enríquez y de oídas lo depuso en 23...,Actos Procesales
1,1,Doña Juana de Fonseca lo depuso en 19 de abr...,Actos Procesales
2,2,"Fray Alonso de Horozco depone de oídas , lo...",Actos Procesales
3,3,Testigoº5 Doña Antonia de Branches depuso a ...,Actos Procesales
4,4,Testigoº No declarado en el proceso Fray Ant...,Actos Procesales
...,...,...,...
1226,1227,Siendo a todo ello testigos Juan Velázquez d...,Roles Procesales
1227,1228,Siendo a todo ello testigos Juan Velázquez d...,Roles Procesales
1228,1229,Siendo a todo ello testigos Juan Velázquez d...,Roles Procesales
1229,1230,Siendo a todo ello testigos Juan Velázquez d...,Roles Procesales


In [23]:
labels_to_ids = {'Actos Procesales':0,'Circunstancial': 1, 'Connivencia': 2, 'Materia Teológica': 3, 'Pertenencia': 4, 'Roles Procesales': 5}
ids_to_labels = {v:k for k, v in labels_to_ids.items()}
labels_list = [x for x in ids_to_labels.values()]
prompted_sentence = [i for i in data['Sentence']]
labels = [i for i in data['Label']]
encoded_labels = [[k] for va in labels for k, v in ids_to_labels.items() if va == v]

In [24]:
tokenizer = AutoTokenizer.from_pretrained(custom_tokenizer_path)
vocab_size = tokenizer.vocab_size 
print(f"Updated tokenizer vocabulary size: {vocab_size} and the len of the tokenizer is {len(tokenizer)}")

Updated tokenizer vocabulary size: 31002 and the len of the tokenizer is 31002


In [6]:
tokenizer.added_tokens_decoder

{0: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 1: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 4: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 5: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 6: AddedToken("[PERSON]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 7: AddedToken("[PERSON_REFERENCE]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 8: AddedToken("[ORG]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 9: AddedToken("[PLACE]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 10: AddedToken("[CONTEXT]", rstrip=False, ls

In [26]:
tokenized_inputs = tokenizer(
            prompted_sentence,
            truncation=True,

            padding=True,

            return_tensors='pt'
        ) # 
tokenized_inputs['labels'] = torch.tensor(encoded_labels)

In [8]:
for i, tensor in enumerate(tokenized_inputs['sequence_positions'][:,0]):
    if min(tensor) < 0 or min(tensor) == 0:
        print(f'index: {i}, max: {max(tensor)}, min: {min(tensor)}')
else: 
    print('all tensors have positive values for the position embedding layer')

all tensors have positive values for the position embedding layer


In [9]:
for i, tensor in enumerate(tokenized_inputs['sequence_positions'][:,1]):
    if min(tensor) < 0 or min(tensor) == 0:
        print(f'index: {i}, max: {max(tensor)}, min: {min(tensor)}')
else: 
    print('all tensors have positive values for the position embedding layer')

all tensors have positive values for the position embedding layer


In [10]:
from torch import nn
import torch
from typing import Optional, Union, Tuple
from torch.nn import CrossEntropyLoss, MSELoss, BCEWithLogitsLoss
from transformers import BertPreTrainedModel, BertModel, BertForSequenceClassification
from transformers.models.bert.modeling_bert import SequenceClassifierOutput
from relation_extraction_encoders_modeling import Classifier_MLP_with_AvgPooling, Classifier_MLP
class SimpleBertLikeForSequenceClassificationWithLSTM(BertPreTrainedModel):
    """
    Simple BERT-like model with LSTM for sequence classification.

    This model integrates BERT embeddings with a bidirectional LSTM layer
    followed by an MLP classifier for sequence classification tasks.

    Parameters:
        config (:obj:`BertConfig`): Configuration class for BERT model.
        num_positional_embeddings (int): Number of positional embeddings.
        positional_embedding_dim (int): Dimensionality of positional embeddings.
        padding_idx (Optional[int]): Index for padding tokens in positional embeddings.
        MLP_hidden_size (int): Hidden size of the MLP classifier.
        Gelu_aproximation (str): Approximation method for GELU activation function.

    Attributes:
        num_labels (int): Number of labels for sequence classification.
        config (:obj:`BertConfig`): Configuration class instance for BERT model.
        bert (:obj:`BertModel`): BERT model without pooling layer.
        positional_embedding (:obj:`nn.Embedding`): Positional embedding layer.
        Bidirectional_block (:obj:`nn.LSTM`): Bidirectional LSTM layer.
        classifier (:obj:`Classifier_MLP`): MLP classifier for final prediction.

    Methods:
        forward(input_ids, attention_mask, token_type_ids, position_ids, head_mask,
                inputs_embeds, labels, output_attentions, output_hidden_states,
                return_dict, sequence_positions):
            Performs forward pass through the model and returns logits or SequenceClassifierOutput.
    """
    def __init__(
            self,
            config,
            MLP_hidden_size: int = 384,
            Gelu_aproximation: str = 'none'
            ):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.bert = BertModel(config, add_pooling_layer=True)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )


        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = Classifier_MLP(
            input_dim=self.config.hidden_size,
            hidden_dim=MLP_hidden_size,
            dropout_rate=classifier_dropout,
            Gelu_aproximation=Gelu_aproximation,
            output_dim=self.num_labels
        )
 
        # Initialize weights and apply final processing
        self.post_init()

    def forward(
            self,
            input_ids: Optional[torch.Tensor] = None,
            attention_mask: Optional[torch.Tensor] = None,
            token_type_ids: Optional[torch.Tensor] = None,
            position_ids: Optional[torch.Tensor] = None,
            head_mask: Optional[torch.Tensor] = None,
            inputs_embeds: Optional[torch.Tensor] = None,
            labels: Optional[torch.Tensor] = None,
            output_attentions: Optional[bool] = None,
            output_hidden_states: Optional[bool] = None,
            return_dict: Optional[bool] = None,

        ) -> Union[Tuple[torch.Tensor], SequenceClassifierOutput]:

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict



        # Get BERT outputs
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

   
        # Extract indices for separation tokens (assuming sep_token_id is 5)
    
        # Separate BERT output into segments
        pooled_output= outputs[1]
   
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        # Check logits for NaNs


        loss = None
        if labels is not None:


            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.squeeze(), labels.squeeze())
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)



        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [27]:
from transformers import BertForSequenceClassification
test_model = BertForSequenceClassification.from_pretrained(
                                                                            custom_model_path,
                                                                            num_labels= len(labels_list),
                                                                            id2label=ids_to_labels,
                                                                            label2id=labels_to_ids
                                                                                )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /home/tensorboard/Documentos/1. D4R/Relation extraction/custom basic model/basic_model and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
with torch.no_grad():
    input_ids = tokenized_inputs['input_ids'][0:10] # Adds batch dimension
    attention_mask = tokenized_inputs['attention_mask'][0:10]
    token_type_ids = tokenized_inputs['token_type_ids'][0:10]

    
    labels = tokenized_inputs['labels'][0:10]
    
    out = test_model(input_ids=input_ids,
                     attention_mask=attention_mask,
                     token_type_ids=token_type_ids,

                     labels=labels)


In [30]:
out

SequenceClassifierOutput(loss=tensor(1.7308), logits=tensor([[ 0.1731,  0.2718,  0.1162, -0.1931, -0.0942,  0.1442],
        [ 0.2471,  0.1847,  0.0234, -0.1588,  0.0077,  0.1365],
        [ 0.3040,  0.3009,  0.0870, -0.1940,  0.0272,  0.1686],
        [ 0.2933,  0.1098, -0.0486, -0.1002,  0.0377,  0.0655],
        [ 0.2259,  0.1885,  0.0542, -0.1926,  0.0890,  0.1612],
        [ 0.2196,  0.4372,  0.0671, -0.1535, -0.1551,  0.2606],
        [ 0.0786,  0.2172, -0.0486, -0.0246, -0.0558,  0.2582],
        [ 0.0786,  0.2172, -0.0486, -0.0246, -0.0558,  0.2582],
        [ 0.0786,  0.2172, -0.0486, -0.0246, -0.0558,  0.2582],
        [ 0.0786,  0.2172, -0.0486, -0.0246, -0.0558,  0.2582]]), hidden_states=None, attentions=None)

In [14]:
from transformers import BertForSequenceClassification

In [14]:
# General variables:



MODEL_SUMMARY_NAME = 'BETO'
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 2e-4
PER_DEVICE_TRAIN_BATCH_SIZE = 4

NUM_TRAIN_EPOCHS = 1
WEIGHT_DECAY = 0.1
EVALUATION_STRATEGY = "steps"
LOGGING_STEPS = 10
SAVE_STRATEGY = "steps"
SAVE_STEPS = 500
SAVE_ONLY_MODEL = True
OUTPUT_DIRECTORY = f"/home/tensorboard/Documentos/1. D4R/8. Models/RE-LSTM-{str(MODEL_SUMMARY_NAME)}-{str(LEARNING_RATE)}-lr-{str(NUM_TRAIN_EPOCHS)}-epochs"
LR_SCHEDULER_TYPE = 'constant'
#['linear', 'cosine', 'cosine_with_restarts', 'polynomial', 'constant', 'constant_with_warmup', 'inverse_sqrt', 'reduce_lr_on_plateau', 'cosine_with_min_lr']

In [42]:
import numpy as np
import evaluate

# Load the evaluation metrics
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    precision_result = precision_metric.compute(predictions=predictions, references=labels)
    recall_result = recall_metric.compute(predictions=predictions, references=labels)
    f1_result = f1_metric.compute(predictions=predictions, references=labels)
    
    return {
        "precision": precision_result["precision"],
        "recall": recall_result["recall"],
        "f1": f1_result["f1"]
    }



In [31]:
datacollator = DataCollatorWithPadding(tokenizer=tokenizer)
dataset = baseDataset(tokenized_inputs)

In [34]:
# Training Process
training_args = TrainingArguments(
    output_dir=r'/home/tensorboard/Documentos/1. D4R/8. Models',
    learning_rate=1e-4 ,
    per_device_train_batch_size=10,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy='steps',
    save_only_model= True,
    lr_scheduler_type='constant',
    logging_steps= 10,


)

trainer = Trainer(
    model=test_model, # type: ignore
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=datacollator,

)



In [35]:
trainer.train()

  0%|          | 0/1240 [00:00<?, ?it/s]

{'loss': 0.9727, 'grad_norm': 27.092010498046875, 'learning_rate': 0.0001, 'epoch': 0.08}
{'loss': 1.3893, 'grad_norm': 5.417557239532471, 'learning_rate': 0.0001, 'epoch': 0.16}
{'loss': 1.2108, 'grad_norm': 5.926936149597168, 'learning_rate': 0.0001, 'epoch': 0.24}
{'loss': 1.0682, 'grad_norm': 5.868988037109375, 'learning_rate': 0.0001, 'epoch': 0.32}
{'loss': 0.7055, 'grad_norm': 5.5226874351501465, 'learning_rate': 0.0001, 'epoch': 0.4}
{'loss': 1.051, 'grad_norm': 5.881610870361328, 'learning_rate': 0.0001, 'epoch': 0.48}
{'loss': 0.8385, 'grad_norm': 2.8503479957580566, 'learning_rate': 0.0001, 'epoch': 0.56}
{'loss': 0.9384, 'grad_norm': 3.5090768337249756, 'learning_rate': 0.0001, 'epoch': 0.65}
{'loss': 0.7734, 'grad_norm': 2.2956290245056152, 'learning_rate': 0.0001, 'epoch': 0.73}
{'loss': 0.7434, 'grad_norm': 5.564837455749512, 'learning_rate': 0.0001, 'epoch': 0.81}
{'loss': 0.8529, 'grad_norm': 8.871514320373535, 'learning_rate': 0.0001, 'epoch': 0.89}
{'loss': 0.8868, '

KeyboardInterrupt: 